In [4]:
!pip install ibm-cos-sdk pandas


In [24]:
import ibm_boto3
from botocore.client import Config
import pandas as pd
import types
import time
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='DxcBbDB4i0U__STHOAiEoe2Wr3y1cXXHIe-HZnjaefxo',
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'bucket-p0xk03358pv9r9k'
object_key = 'college_feedback.csv'


body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

def _iter_(self): return 0
if not hasattr(body, "_iter_"):
    body._iter_ = types.MethodType(_iter_, body)

data = pd.read_csv(body)


credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "9RPbPj8PN5Lddp_yYc91QDQH9n9dh8OIwioIqABZHpq1"
}
project_id = "297411de-d1b7-49ca-ac8a-60715dcff4db"

model = Model(
    model_id="google/flan-t5-xxl",
    params={ GenParams.MAX_NEW_TOKENS: 10 },
    credentials=credentials,
    project_id=project_id
)

few_shot = """
Classify the feedback into one of these categories:
Academics, Facilities, Administration

Examples:
Feedback: The professors explain concepts very clearly.
Category: Academics

Feedback: The syllabus is too vast to cover in one semester.
Category: Academics

Feedback: The classroom is always clean and well maintained.
Category: Facilities

Feedback: The canteen food is not hygienic.
Category: Facilities

Feedback: The administration staff is friendly and helpful.
Category: Administration

Feedback: There are frequent delays in issuing ID cards.
Category: Administration

Feedback: {feedback}
Category:
"""

results = []
for fb in data['Feedback']:
    prompt = few_shot.replace("{feedback}", fb)
    try:
        response = model.generate_text(prompt=prompt)
        category = "Error"
        if isinstance(response, dict):
            if "results" in response and len(response["results"]) > 0:
                category = response["results"][0]["generated_text"].strip()
            elif "generated_text" in response:
                category = response["generated_text"].strip()
        elif isinstance(response, str):
            category = response.strip()

        results.append(category)
    except Exception:
        results.append("Error")
    time.sleep(1)  


data['Predicted Category'] = results
filename = f"college_feedback_classified_{int(time.time())}.csv"
data.to_csv(filename, index=False)


cos_client.upload_file(Filename=filename, Bucket=bucket, Key=filename)
print(f"Results uploaded to COS as {filename}")


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


Results uploaded to COS as college_feedback_classified_1750785304.csv
